### Practice SQL queries with Python and sqlite3

As a data scientist, you'll not only be analysing, but assembling data into larger datasets. While you could add individual rows to a table, its more common to add many rows, or even create a new table from a large file. Comma separated variable (CSV) files are simply text with the column entries in each row separated by a comma. Other separators can be used instead, but let's stick with commas. 
There is an abundance of data on the internet, but let's make some ourselves. We will create data to populate some fields in two CSV files. Let's imagine that we work for Star Force (I was going to use "Space Force" until I learned there is already an organization called that. Can you believe that?). Star Force command suspects that its best starship pilots, the ones who have accumulated the highest scores, are teamed with the best engineers. Engineers are shipmates who aid in all manner of starship systems control and maintenance. Command has asked you, a newly assigned analyst, to confirm their belief; they have asked you to identify the engineers teamed with the highest performing pilots, say with scores above 0.85.
First, let's create two CSV files: pilots.csv and engineers.csv. Then we create a database and insert the pilots data into one table, and insert the engineers data into the other.

In [1]:
import csv, sqlite3

In [2]:
with open('./data/pilots.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    rows = []
    FIRSTROW = 1
    for row in readCSV:
        if not FIRSTROW:
            if len(row)>0:
                rows.append(row)
        else:
            FIRSTROW = 0

conn = sqlite3.connect('./data/starforce_personnel.db')
cur = conn.cursor()

# Create table
cur.execute('''CREATE TABLE pilots
             (forceId integer, name text, score real, shipmateId integer)''')

for irow in range(len(rows)):
    # Insert a row of data
    thisRow = rows[irow]
    cur.execute("INSERT INTO pilots VALUES (?,?,?,?)", thisRow)


# commit (save) the changes and close the connection
conn.commit()
conn.close()

Note we must first create a Connection object that represents the database. Then we can create a Cursor object and call its execute() method to perform SQL commands. In the CREATE TABLE command above, you'll notice that for each column name we define, we must speficy the data type. "forceId" is an integer, "name" is text, "score" is real (floating-point), and shipmateId is also of type integer.
Let's print this first table to see what kind of data we are dealing with. I first print the column names before printing the rows, and I've defined a spacing for each column to make the table easier to read.

In [3]:
conn = sqlite3.connect('./data/starforce_personnel.db')
cur = conn.cursor()

# get column names and 
cur.execute("select * from pilots where 1=0;")
col_names = [x[0] for x in cur.description]
print('\n',f'{col_names[0]:<8} | {col_names[1]:<11} | {col_names[2]:<8} | {col_names[3]}')
print('-'*(11+14+12+8))

# query to get all rows
cur.execute("select * from pilots")
for row in cur:
    rowstr = f'{row[0]:<11}{row[1]:<14}{row[2]:<12}{row[3]}'
    print(' ' + rowstr)

conn.close()


 forceId  | name        | score    | shipmateId
---------------------------------------------
 1001       Xerxoid       0.978       700
 992        Memoisma      0.777       711
 223        Uhrlohna      0.885       723
 536        Schwifty      0.996       776
 109        Xuma          0.899       


It seems that Xuma has no shipmate. Maybe his shipmate was transferred, or maybe he is flying an experimental class fighter that requires only a pilot. As data analysts and data scientists, we must get used to missing entries like this. Now let's read the engineers data from file and save this to a separate table in our database.

In [4]:
with open('./data/engineers.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    rows = []
    FIRSTROW = 1
    for row in readCSV:
        if not FIRSTROW:
            if len(row)>0:
                rows.append(row)
        else:
            FIRSTROW = 0

conn = sqlite3.connect('./data/starforce_personnel.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE engineers
             (forceId integer, name text, score real, shipmateId integer)''')

for irow in range(len(rows)):
    # Insert a row of data
    thisRow = rows[irow]
    c.execute("INSERT INTO engineers VALUES (?,?,?,?)", thisRow)

conn.commit()
conn.close()

Let's read the engineers table.

In [5]:
conn = sqlite3.connect('./data/starforce_personnel.db')
cur = conn.cursor()

# get column names and 
cur.execute("select * from engineers where 1=0;")
col_names = [x[0] for x in cur.description]
print('\n',f'{col_names[0]:<8} | {col_names[1]:<11} | {col_names[2]:<8} | {col_names[3]}')
print('-'*(11+14+12+8))

# query to get all rows
cur.execute("select * from engineers")
for row in cur:
    rowstr = f'{row[0]:<11}{row[1]:<14}{row[2]:<12}{row[3]}'
    print(' ' + rowstr)

conn.close()


 forceId  | name        | score    | shipmateId
---------------------------------------------
 723        Qualan        0.57        223
 776        Yhrr          0.878       536
 700        Osann-qui     0.902       1001
 711        Siratik       0.917       992
 784        Qyllichk      0.942       437
 767        Phyrhynn      0.965       692


Wait a sec, there are more entries in the engineers table than in the pilots table. That must mean that some of the engineers shipmates are not listed in the pilots table. We need to do a query to find out which engineers shipmateId values are not found the in pilots forceId column. This is getting ahead of ourselves though since we haven't even learned to do a simple query. Lets do that now by checking which pilots have scores above 0.85.

In [6]:
conn = sqlite3.connect('./data/starforce_personnel.db')
cur = conn.cursor()
query_1 = '''
SELECT forceId, name
FROM pilots
WHERE pilots.score > 0.85;
'''
cur.execute(query_1)
for row in cur:
    print(row)

(1001, 'Xerxoid')
(223, 'Uhrlohna')
(536, 'Schwifty')
(109, 'Xuma')


Great, we can now SELECT column1, column2, etc., FROM a table specifying a filter using WHERE. We dispensed with the pretty printing here and notice how few lines of code are required for a query. Now let's try to answer the question that was bugging us: Which engineers shipmateId values are not found in the pilots forceId column?

In [7]:
conn = sqlite3.connect('./data/starforce_personnel.db')
cur = conn.cursor()
query_2 = '''
SELECT shipmateId
FROM   engineers
WHERE  shipmateId NOT IN (SELECT forceId FROM pilots)
'''
cur.execute(query_2)
for row in cur:
    print(row)

(437,)
(692,)


Looking back where we printed the pilots table we can verify that indeed there are no pilots with these two forceId values. Some overworked Star Force official somehow forgot to add this data. We'll worry about that later, for now we are getting confident in our SQL and can perfom a query to answer the question that Star Force Command gave us, namely, "What are the names of the pilot & engineer teams with pilot scores greater than 0.85?".

In [8]:
# INNER JOIN with filter
query_3 = '''
SELECT engineers.name, pilots.name
FROM
    engineers
        INNER JOIN
    pilots ON pilots.shipmateId = engineers.forceId
WHERE pilots.score > 0.85;
'''
cur.execute(query_3)
for row in cur:
    print(row)

('Osann-qui', 'Xerxoid')
('Qualan', 'Uhrlohna')
('Yhrr', 'Schwifty')


This query was more advanced that the previous one, using something called INNER JOIN, so we should check the result carefully. Compare the two printed tables, you'll see that Xerxoid, Uhrlohna, and Schwifty were the three pilots having scores above 0.85, and that Osann-qui, Qualan, and Yhrr are their respective engineers. Xuma also has a score above 0.85, however he has no assigned engineer and so can't be considered here.

This data and these examples were contrived to be simple, but I hope that they illustrate how easily a database can be read from and written to with Python & sqlite3. Later, I'll introduce more complex queries on larger datasets, trying to cover the most common types that a data scientist might use, or that an aspiring data scientist might expect to encounter in a technical interview.